In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, cross_validation
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('data.csv')
sample = data

In [8]:
cat_features = ['pitcher_id', 'batter_id', 'away_team', 'home_team', 
                'year', 'b_stand', 'p_throws', 'inning_half',  
                'batter_team', 'pitcher_team', 'type']
real_features = ['night', 'inning', 'order', 'home', 'weekday', 
                 'month', 'balls', 'strikes', 'sz_top', 'sz_bot']
n_cat = len(cat_features) - 1 # -1 for type
n_real = len(real_features)
ptypes = sample.type.unique()
depths = [len(data[col].unique()) for col in cat_features]

In [9]:
N = len(sample)
n_outputs = len(ptypes)
embedding_size = 30
prep = learn.preprocessing.CategoricalProcessor()
cat_data = np.array(list(prep.fit_transform(sample[cat_features])))
n_pitchers = cat_data[:,0].max() + 1
n_batters = cat_data[:,1].max() + 1

real_data = sample[real_features].values
result_data = cat_data[:,-1]
cat_data = cat_data[:,:-1]

In [13]:
p_embeddings = tf.Variable(tf.random_uniform([n_pitchers, embedding_size], -1.0, 1.0))
b_embeddings = tf.Variable(tf.random_uniform([n_batters, embedding_size], -1.0, 1.0))

cat_batch = tf.placeholder(tf.int32, [None, n_cat])
real_batch = tf.placeholder(tf.float32, [None, n_real])
result_batch = tf.placeholder(tf.int32, [None])

pitchers = cat_batch[:,0]
batters = cat_batch[:,1]

p_embed = tf.nn.embedding_lookup(p_embeddings, pitchers)
b_embed = tf.nn.embedding_lookup(b_embeddings, batters)

inputs = [p_embed, b_embed, real_batch]
for i in range(2, n_cat):
    inputs.append(tf.one_hot(cat_batch[:,i], depth=depths[i],
                  on_value=1.0, off_value=0.0, dtype=tf.float32))
input_layer = tf.concat(concat_dim=1, values=inputs)

in_dim = input_layer.get_shape()[1]

W1 = tf.Variable(tf.zeros([in_dim, 75]))
b1 = tf.Variable(tf.zeros([75]))

hidden = tf.nn.relu(tf.matmul(input_layer, W1) + b1)

W2 = tf.Variable(tf.zeros([75, n_outputs]))
b2 = tf.Variable(tf.zeros([n_outputs]))

y = tf.nn.softmax(tf.matmul(hidden, W2) + b2)
y_ = tf.one_hot(indices=result_batch, depth=n_outputs, 
                on_value=1.0, off_value=0.0, dtype=tf.float32)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.initialize_all_variables()

In [22]:
sess = tf.InteractiveSession()
sess.run(init)
batch_size = 100
for i in range(10000):
    idx = np.random.randint(0, data.shape[0], batch_size)
    input_data = feed_dict={cat_batch : cat_data[idx],
                                        real_batch : real_data[idx], 
                                        result_batch : result_data[idx]}
    sess.run(train_step, feed_dict=input_data)
    if i % 100 == 0:
        print(sess.run(cross_entropy, feed_dict=input_data))

2.2156
1.84501
1.87403
2.0088
1.78635
1.86596
1.98808
1.86645
1.95474
1.89324
1.88269
1.81515
1.90046
1.86563
1.96399
1.94041
1.98976
1.88657
1.92241
1.86227
1.95941
1.92507
1.96407
1.78848
1.92578
1.77164
1.9423
1.90178
1.86012
1.92233
1.89106
1.89043
1.79072
1.91971
1.85136
1.97118
1.93519
1.88611
1.8199
1.84727
1.83663
1.93284
1.72864
1.75813
1.81995
2.06889
1.79074
1.76029
1.92533
1.9499
1.89887
1.96366
1.91524
1.77349
1.86969
1.931
1.89294
2.01398
1.90457
1.88455
1.75282
1.78378
1.823
1.9278
1.81076
1.99548
1.9111
1.82358
1.84163
1.97825
1.73871
1.82709
1.90067
1.82791
1.8393
1.81595
1.90053
1.91563
1.80612
1.71503
1.99787
1.85574
1.85874
1.75967
1.83699
1.74668
1.82854
1.83684
1.91736
1.86353
1.90647
1.90214
1.91601
1.89341
1.81655
1.92413
1.84993
1.82894
1.82711
1.82604
